In [ ]:
import pandas as pd
import numpy as np
from contextlib import contextmanager
import os,sys,datetime, time
from sklearn.metrics import roc_auc_score
from sklearn import linear_model
from sklearn import svm, neighbors

@contextmanager
def timer(name):
    """
    Taken from Konstantin Lopuhin https://www.kaggle.com/lopuhin
    in script named : Mercari Golf: 0.3875 CV in 75 LOC, 1900 s
    https://www.kaggle.com/lopuhin/mercari-golf-0-3875-cv-in-75-loc-1900-s
    """
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

kfold = 4
DataBaseDir = '../../data/version2'
InputDir = '%s/l1' % DataBaseDir
OutputDir = '%s/l2' % DataBaseDir
targets = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
strategy = 'l1-knn'
# meta_models = ['nbsvm', 'bi-gru', 'lr', 'bi-gru-cnn', 'lgb', 'fm-ftrl', 
#                'bi-lstm', 'bi-gru-glove', 'cnn']#, 'cnn-num-glove']#, 'bi-gru-attention-glove']
# meta_models = ['bi-gru', 'bi-gru-cnn','fm-ftrl', 'lgb', 'lr', 'nbsvm', 'cnn', 'bi-gru-attention',
#                'bi-lstm-attention', 'bi-gru-capsule', 'bi-lstm-cnn-2-v1']#, 'cnn-num-glove', 'bi-lstm']
meta_models = ['fm-ftrl', 'lgb', 'nbsvm', 'lr', 'bi-gru-cnn','bi-gru', 
               'cnn', 'bi-gru-capsule', 'bi-gru-attention', 'bi-gru-cnn-glove']#, 'cnn-wiki', 'cnn-twitter-200']#,'bi-lstm']#, 'cnn-num']
drop_cols = ['id', 'fold']

# load data
num_cols = []
valid_dfs = []
test_dfs = []
with timer('load data'):
    for fold in range(kfold):
        FoldInputDir = '%s/kfold/%s' % (InputDir, fold)
        for i in range(len(meta_models)):
            valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, meta_models[i])).reset_index(drop= True)
            test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, meta_models[i])).reset_index(drop= True)
            if(i == 0):
                FoldValid = valid
                FoldTest = test
            else:
                for t in targets:
                    target = '%s_%s' % (meta_models[i], t)
                    FoldValid[target] = valid[target].copy()
                    FoldTest[target] = test[target].copy()

#         ##
#         NumFoldInputDir = '%s/num/kfold/%s' % (DataBaseDir, fold)
#         NumValid = pd.read_csv('%s/valid.csv' % NumFoldInputDir).reset_index(drop= True)
#         TmpNumCols = []
#         for c in NumValid.columns:
#             tmp_drop_cols = drop_cols.copy()
#             tmp_drop_cols.extend(targets)
#             if(c not in tmp_drop_cols):
#                 FoldValid[c] = NumValid[c]
#                 TmpNumCols.append(c)
#         num_cols = TmpNumCols
#         NumTest = pd.read_csv('%s/test.csv' % NumFoldInputDir).reset_index(drop= True)
#         for c in NumTest.columns:
#             tmp_drop_cols = drop_cols.copy()
#             tmp_drop_cols.extend(targets)
#             if(c not in tmp_drop_cols):
#                 FoldTest[c] = NumTest[c]

        FoldValid['fold'] = fold
        valid_dfs.append(FoldValid)
        test_dfs.append(FoldTest)
        print('fold %s done.' % fold)
    TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
    print('load data done, train %s' % (len(TrainData)))

## CV score
cv_score = .0
# predict
y_test_pred = 0
pred_targets = ['%s_%s' % (strategy, t) for t in targets]
TestData = test_dfs[0][['id']].copy()
for t in pred_targets:
    TestData[t] = .0
# parameter_options = {
#     'alpha': [0.04, 0.004, 0.0004],
#     'l1_ratio': [0.1, 0.4, 0.8]
# }
for fold in range(kfold):
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'test': test_dfs[fold].copy()
    }
    for t in targets:
        FoldData['valid']['%s_%s' % (strategy, t)] = .0
        FoldData['test']['%s_%s' % (strategy, t)] = .0
    
    with timer('training'):
        for target in targets:
            meta_feats = ['%s_%s' % (c, target) for c in meta_models]
#             meta_feats.extend(num_cols)
            # train
            model = neighbors.KNeighborsClassifier(n_neighbors= 512)
            model.fit(FoldData['train'][meta_feats], FoldData['train'][target])
            # for valid
            pred_target = '%s_%s' % (strategy, target)
            FoldData['valid'][pred_target] = model.predict_proba(FoldData['valid'][meta_feats])[:,1]
            # for test
            pred_test = model.predict_proba(FoldData['test'][meta_feats])[:,1]
            FoldData['test'][pred_target] = pred_test
            TestData[pred_target] += pred_test
            
#             print(FoldData['valid'][[pred_target, target]].sum(axis= 0))
#     print(len(FoldData['valid']))
    
    score = roc_auc_score(FoldData['valid'][targets], FoldData['valid'][pred_targets])
    cv_score += score

    ## output
    FoldOutputDir = '%s/kfold/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in ['valid', 'test']:
        if(mod == 'test'):
            out_cols = ['id']
            out_cols.extend(pred_targets)
        else:
            out_cols = pred_targets.copy()
            out_cols.extend(targets)
        FoldData[mod][out_cols].to_csv('%s/%s_%s.csv' % (FoldOutputDir, mod, strategy),float_format='%.8f', index= False) 
    print('fold %s, score %.5f' % (fold, score))

for t in pred_targets:
    TestData[t] /= kfold
cv_score /= kfold # Average valid set predictions
print('\n================')
print('cv score %.5f' % (cv_score))
print('================')

## submit
sub = TestData[['id']].copy()
sub[targets] = TestData[pred_targets]
OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/submit' % InputDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.8f', index=False)
print('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))
os.system('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))

fold 0 done.
fold 1 done.
fold 2 done.
fold 3 done.
load data done, train 159571
[load data] done in 14 s


/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
